In [ ]:
# data 灰色图片 1 channel 彩色图片RGB 32*32
# batch 批度- 数据量 
# input shape维度 图片中经常用到的 batch*channel*height*width 1*1*32*32


In [41]:
# 包
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
'''
卷积
stride 步长 1
padding 补0 0
kernel_size 3 (3,3)  (4,3)
channel 核的数目 
池化
mean-pool 
max-pool
stride = kernel_size 
'''

In [50]:
# 类 卷积
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 
        self.conv1 = nn.Conv2d(1,6,5)
        # nn.MaxPool2d
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5 ,120) # linear 最后一维
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10) # output_dim 
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2)) 
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2)) # 1 *  16*5*5   1*400
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [46]:
image = torch.randn(1,1,32,32)
image

tensor([[[[-2.5809, -2.2491, -0.0456,  ..., -0.8281,  0.2900,  0.1445],
          [ 0.4924, -0.4074, -0.5083,  ...,  0.6781, -0.6279,  0.3205],
          [-0.8451,  1.1822, -1.0632,  ..., -0.4028,  0.6121, -1.3489],
          ...,
          [-0.5405, -0.6952, -1.8725,  ...,  1.0007, -0.5759, -0.9925],
          [-0.7152, -0.8450, -0.4680,  ...,  0.3789,  0.5160, -0.6079],
          [ 0.0461, -0.2224,  0.1851,  ...,  1.5589,  0.9761, -0.7375]]]])

In [47]:
self_conv1 = nn.Conv2d(1,6,5)
self_conv1(image).shape
F.relu(self_conv1(image)).shape
F.max_pool2d(F.relu(self_conv1(image)),(2,2)).shape


torch.Size([1, 6, 28, 28])

torch.Size([1, 6, 28, 28])

torch.Size([1, 6, 14, 14])

In [59]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [53]:
for param in list(net.parameters()):
    print(param.size())

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [60]:
image = torch.randn(1,1,32,32)
out = net(image)
print(out)

tensor([[-0.1538,  0.2345,  0.0163, -0.0747, -0.0048, -0.0948,  0.0715,  0.1349,
         -0.1025,  0.0689]], grad_fn=<AddmmBackward>)


In [55]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [58]:
net.zero_grad()
print('before backward')
print(net.conv1.bias.grad)

out.backward(torch.randn(1,10))
print('after backward')
print(net.conv1.bias.grad)

before backward
None
after backward
tensor([ 0.0020,  0.0031,  0.0231, -0.0334,  0.0158,  0.0332])


In [61]:
# 前向  loss标准 反向
# predict y - truth y 差值 距离 均方误差 绝对值误差 交叉熵 多分类中
target = torch.randn(1,10)

criterion = nn.MSELoss()
loss = criterion(out,target)
print(loss)

tensor(0.6358, grad_fn=<MseLossBackward>)


In [ ]:
# 求梯度 以及 优化/更新 分开
loss.backward()
weight = weight - learning_rate * gradient 
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(learning_rate * f.grad.data)


In [62]:
import torch.optim as optim # SGD Momentum Adam
optimizer = optim.SGD(net.parameters(),lr = 0.01)

In [63]:
optimizer.zero_grad()
out = net(image)
loss = criterion(out,target)
loss.backward()
optimizer.step()

tensor(0.6358, grad_fn=<MseLossBackward>)
